# Assigning genus and species IDs based on pyANI analysis run on each connected component

**Set Up**

In [18]:
import pandas as pd
from pathlib import Path
import numpy as np
import networkx as nx
import dwave_networkx as dnx

**Loading data and prepartaion**
Here we are:
- loading the `pyani report` output, which contains information like run ID, and run name etc.
- getting dictionary with run names keyed by run ID

In [19]:
data = pd.read_csv(Path("../output/runs.tab"), sep='\t')
data.rename(columns = {'run ID':'run_ID', 'date run':'date_run'}, inplace = True)

In [20]:
run_ID_name = data.set_index("run_ID").to_dict()['name']

**Assigning species and genus ID**
Here, we:
- write a function that removes suffix provided in the pyANI matrices
- get a function that will use provided dataframe to calculate MST, and remove edges between genomes if their thereshold is lower than the one provided. In case, where cliques are present it will keep removing the lowest weight until no cliques are present. 

In [21]:
def remove_suffix(row, col):
    """Return accession number from 'FILE' column."""

    try:
        new = ' '.join(row[col].split(':')[:-1])
    except IndexError:
        new = 'NA'

    return new



In [22]:
def assign_ANI_tax_ID(df, thereshold, attribute, ANI_ID):
        """Assign taxon IDs based on ANI analysis. 
        
        :param df: dataframe with each row providing pyANI identity and coverage for a given parir of genomes
        :param thereshold: thereshold at which genomes should be separated (num)
        :param attribute: comparision type deciding which should be used to separate genomes identity or coverage
        :param ANI_ID: current number of assigned IDs
        """
        
        #Generate NetworkX grah with identity and coverage as edge attibutes 
        G_comp=nx.from_pandas_edgelist(df, 'genome1', 'genome2', ['identity', 'coverage'])
        
        current_assignments = {} #Hold empty dictionary

        ANI_ID = ANI_ID

        #Remove edges if thereshold for a given attribute is lower 
        edges_to_remove = [(n1,n2) for n1, n2, attrs in G_comp.edges(data=True) if attrs[attribute] < thereshold]
        G_comp.remove_edges_from(edges_to_remove)
        
        #Check if the components are clique, if not remove edges until clique is achived
        components = [_ for _ in list(nx.connected_components(G_comp))]
        for component in components:
            while dnx.is_clique(G_comp, component) == False:
                weights = sorted(list(set([attrs[attribute] for n1, n2, attrs in G_comp.edges(data=True) if n1 in component and n2 in component])))
                edges_to_remove = [(n1,n2) for n1, n2, attrs in G_comp.edges(data=True) if attrs[attribute] < (weights)[1] and n1 in component and n2 in component]
                G_comp.remove_edges_from(edges_to_remove)
                weights.remove(weights[1])

                components = [_ for _ in list(nx.connected_components(G_comp))]
                for component in components:
                    if dnx.is_clique(G_comp, component) == True:
                        break
        #Assign IDs
        components = [_ for _ in list(nx.connected_components(G_comp))]
        for component in components:
            current_assignments.update({_:ANI_ID for _ in component})
            ANI_ID += 1

    
                
        return current_assignments

**Running functions**
Here we:
- generate dataframes for each pyANI run so that it can be taken by `assign_ANI_tax_ID` and assign IDs

In [37]:
current_species_ID = 1
current_genus_ID = 1
genome_genus_ID = {}
genome_species_ID = {}
for run_ID, run_name in run_ID_name.items():
    if run_ID <=116:

        identity = pd.read_csv(Path(f"../output/pyani_matrices_connected_components/matrix_identity_{run_ID}.tab"), sep='\t').rename(columns={'Unnamed: 0': 'genome1'})
        coverage = pd.read_csv(Path(f"../output/pyani_matrices_connected_components/matrix_coverage_{run_ID}.tab"), sep='\t').rename(columns={'Unnamed: 0': 'genome1'})
                        #Melting data
        identity_melt = pd.melt(identity, id_vars=['genome1'], value_vars=[_ for _ in identity if _ != 'genome1'], var_name='genome2', value_name='identity')
        coverage_melt = pd.melt(coverage, id_vars=['genome1'], value_vars=[_ for _ in identity if _ != 'genome1'], var_name='genome2', value_name='coverage')

                        #Combine data
        combined = pd.merge(identity_melt, coverage_melt,  how='left', left_on=['genome1', 'genome2'], right_on = ['genome1','genome2'])
        combined = combined[combined['genome1'] != combined['genome2']] #Remove self-to-self comparisions
                        #Remove duplicate comparisions and keep minimum coverage and average identity
        combined[['genome1','genome2']] = np.sort(combined[['genome1','genome2']].to_numpy(),axis=1)
        fixed = (combined.groupby(['genome1','genome2']).agg(identity = ('identity','mean'), coverage = ('coverage','min')).reset_index())
        fixed["genome1"] = fixed.apply(remove_suffix,col='genome1', axis=1)
        fixed['genome2'] = fixed.apply(remove_suffix,col='genome2', axis=1)
        fixed = fixed.round(2)

                        #Getting Genus ID
        coverage = assign_ANI_tax_ID(fixed, 0.50, 'coverage', current_genus_ID)
        current_genus_ID += len(set(coverage.values()))
        genome_genus_ID.update(coverage)
                        #Getting Species ID
        identity = assign_ANI_tax_ID(fixed, 0.95, 'identity', current_species_ID)
        current_species_ID += len(set(identity.values()))
        genome_species_ID.update(identity)



Adding the information to `~/Desktop/Kiepas_et_al_2023_MLST/data/MLST_scheme_revision/Genomes_ST_info.csv`

In [38]:
data2 = pd.read_csv(Path("../../supplementary_file_8/output/Genome_ST_info.csv").expanduser())

In [39]:
data2['pyani_genus_ID'] = data2['pyani_label'].map(genome_genus_ID).fillna(0)
data2['pyani_species_ID'] = data2['pyani_label'].map(genome_species_ID).fillna(0)

Assign pyANI genus and species IDs to connected components which were not included in the pyANI analysis (eg. singlenton STs).

In [40]:
genus_ID_per_genome = data2.set_index('accession').to_dict()['pyani_genus_ID']

In [41]:
current_genus = int(max(genus_ID_per_genome.values()))
for accession, genus_ID in genus_ID_per_genome.items():
    if genus_ID == 0:
        current_genus += 1
        genus_ID_per_genome[accession] = current_genus
    else:
        genus_ID_per_genome[accession] = int(genus_ID)
    

In [42]:
data2['pyani_genus_ID'] = data2['accession'].map(genus_ID_per_genome).fillna(0)

In [43]:
species_ID_per_genome = data2.set_index('accession').to_dict()['pyani_species_ID']

In [44]:
current_species = int(max(species_ID_per_genome.values()))
for accession, species_ID in species_ID_per_genome.items():
    if species_ID == 0:
        current_species += 1
        species_ID_per_genome[accession] = current_species
    else:
        species_ID_per_genome[accession] = int(species_ID)

In [45]:
data2['pyani_species_ID'] = data2['accession'].map(species_ID_per_genome).fillna(0)

In [46]:
print(max([_ for _ in data2["pyani_species_ID"]]))

295


In [47]:
data2.to_csv(Path("../../supplementary_file_8/output/Genome_ST_info.csv").expanduser(), index=False)

In [48]:
print(data2)

      ST        accession                        organism  16S_copies  \
0      2  GCF_016906245.1       Streptomyces californicus           6   
1      2  GCF_000715745.1       Streptomyces californicus           1   
2      2  GCF_000717645.1       Streptomyces californicus           1   
3      2  GCF_000717965.1  Streptomyces purpeochromogenes           1   
4      2  GCF_000718245.1       Streptomyces californicus           1   
..   ...              ...                             ...         ...   
868  801  GCF_900119365.1            Streptomyces atratus           1   
869  802  GCF_900171555.1       Streptomyces albidoflavus           1   
870  803  GCF_900206255.1                Streptomyces sp.           4   
871  804  GCF_900215595.1                Streptomyces sp.           6   
872  805  GCF_900215615.1                Streptomyces sp.           6   

            strain  assembly_status               status_tax Type_Strain  \
0    FDAARGOS_1213  Complete Genome            